In [ ]:
from pathlib import Path
from PIL import Image
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary
from torchvision import datasets, transforms

In [ ]:
# 1. Configuration and Data Loading
BATCH_SIZE = 64

data_transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
class SingleImageFolder(torch.utils.data.Dataset):
    def __init__(self, root, transform=None):
        self.paths = list(Path(root).glob("*.jpg"))
        self.transform = transform
        self.classes = sorted(list(set(map(self.get_label, self.paths))))

    @staticmethod
    def get_label(path):
        return str(path.with_suffix("")).split('\\')[-1]

    def load_image(self, index):
        image_path = self.paths[index]
        return Image.open(image_path)

    def __len__(self):
        return len(self.paths)

    def __getitem__(self, index):
        img = self.load_image(index)
        class_name = self.get_label(self.paths[index])
        class_idx = self.classes.index(class_name)

        if self.transform:
            return self.transform(img), class_idx
        else:
            return img, class_idx

In [ ]:
train_data = datasets.ImageFolder(
    root='./data/asl_alphabet_train',
    transform=data_transform
)
test_data = SingleImageFolder(
    root="./data/asl_alphabet_test",
    transform=data_transform
)

In [ ]:
train_loader = DataLoader(dataset=train_data, batch_size=BATCH_SIZE,
                              shuffle=True, num_workers=4)
test_loader = DataLoader(dataset=test_data, batch_size=BATCH_SIZE,
                             shuffle=True, num_workers=4)

In [ ]:
for data, target in test_loader:
    print()

In [ ]:
# Use GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

In [ ]:
# 2. Define the Neural Network Model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()
        # 200 200
        self.pool0 = nn.MaxPool2d(kernel_size=4) # 50 50

        self.conv1 = nn.Conv2d(in_channels=3, out_channels=2, kernel_size=3)  #48 48
        self.pool1 = nn.MaxPool2d(kernel_size=2)  # 24 24

        self.conv2 = nn.Conv2d(in_channels=2, out_channels=4, kernel_size=3)  #22 22
        self.pool2 = nn.AdaptiveAvgPool2d(output_size=(3, 3))
        # self.pool2 = nn.MaxPool2d(kernel_size=3)

        self.fc3 = nn.Linear(4 * 3 * 3, 29)

    def forward(self, x):
        x = self.pool0(x)

        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = x.view(-1, 4 * 3 * 3)
        x = self.fc3(x)
        # Apply log softmax for the final output (often used with NLLLoss, which is part of CrossEntropyLoss)
        return F.log_softmax(x, dim=1)


model = SimpleNN().to(device)

In [ ]:
# 3. Define Loss and Optimizer
criterion = nn.NLLLoss()  # Negative Log-Likelihood Loss (suitable for log_softmax output)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:
summary(model)

In [ ]:
from datetime import datetime

writer = SummaryWriter(f'runs/init_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}')

In [ ]:
# 4. Training Loop
NUM_EPOCHS = 5

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()  # Clear gradients
        output = model(data)  # Forward pass
        loss = criterion(output, target)  # Calculate loss
        loss.backward()  # Backward pass (calculate gradients)
        optimizer.step()  # Update weights

        running_loss += loss.item()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch + 1}/{NUM_EPOCHS} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    avg_train_loss = running_loss / len(train_loader)
    writer.add_scalar('Loss/Train', avg_train_loss, epoch)

writer.flush()
writer.close()

In [ ]:
model.eval()

In [ ]:
model.cpu()

In [ ]:
# tensorboard --logdir=runs

In [ ]:
# 5. Testing the Model
test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
accuracy = 100. * correct / len(test_loader.dataset)

print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
      f'({accuracy:.2f}%)\n')

In [ ]:
# Save the model (optional)
torch.save(model.state_dict(), "mnist_simple_nn.pth")

In [ ]:
def export_onnx(model, onnx_path="dynamic_cnn.onnx"):
    dummy = torch.randn(1, 1, 256, 256)

    with torch.inference_mode():
        torch.onnx.export(
            model,
            dummy,
            onnx_path,
            opset_version=18,
            dynamo=False,
            export_params=True,
            do_constant_folding=True,
            input_names=["input"],
            output_names=["output"],
            dynamic_axes={
                "input": {0: "batch", 2: "height", 3: "width"},
                "output": {0: "batch"},
            },
        )

In [ ]:
onnx_path = "model.onnx"
export_onnx(model, onnx_path)

In [ ]:
import onnx

onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)  # Check for model correctness

print("Model inputs:")
for input in onnx_model.graph.input:
    print(f"Input name: {input.name}")
    print(
        f"Input shape: {[(d.dim_value if d.HasField('dim_value') else d.dim_param) for d in input.type.tensor_type.shape.dim]}")

In [ ]:
# Take a look at model graph
# https://netron.app/